# 实体对齐

In [1]:
include("../src/CCKS-mKGQA.jl")
include("../src/loaddata/extractdata.jl")
include("../src/loaddata/translatedata.jl")
include("../src/tools/ettalign.jl")
include("../src/datatype.jl")
# mkpath.("EA_data/" .* (["", "ILLs_1", "ILLs_2", "ILLs_3", "ILLs_4", "ILLs_official", "ILLs_wiki"]))

### 生成数据
ILLs => `train_ques, train_data, valid_ques, triples, equivills`

In [ ]:
illpath = "ILLs_1"
# illpath = "ILLs_2"
# illpath = "ILLs_3"
# illpath = "ILLs_4"
# illpath = "ILLs_official"
# illpath = "ILLs_wiki"
include("../src/loaddata/illsdata.jl")
equivills_values = unique!(sort!(collect(values(illsdata.equivILLs)); by=length, rev=true));

### 测试

In [47]:
subs = unique!(first.(illsdata.ills_triples))
edges = illsdata.ills_edges
# 第一跳
jumps = Dict(sub => [length(edges[sub])] for sub in subs)
# 第二跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][1] for (rel, obj) in edges[sub]))
end
# 第三跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][2] for (rel, obj) in edges[sub]))
end

In [39]:
# ILLs 4
println(illsdata.illsname)
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

ILLs_4
[0, 24311, 31355]


(23, 73)

In [42]:
# ILLs 3
println(illsdata.illsname)
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

ILLs_3
[0, 37283, 48256]


(61, 137)

In [36]:
# 官方 ILLs
println(illsdata.illsname)
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

ILLs_official
[0, 52268, 67437]


(156, 352)

In [48]:
# ILLs 1
println(illsdata.illsname)
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

ILLs_1
[0, 44396, 57285]


(80, 209)

In [45]:
# ILLs 2
println(illsdata.illsname)
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

ILLs_2
[0, 47455, 61439]


(98, 263)

In [32]:
# wiki ILLs
println(illsdata.illsname)
println([count(iszero, getindex.(values(jumps), i)) for i in 1:3])
count(i->jumps[i][2]==0, ill_valid_ners), count(i->jumps[i][3]==0, ill_valid_ners)

ILLs_wiki
[0, 38070, 49921]


(6, 103)

### 保存

In [49]:
open("EA_data/$illpath/triples.txt", "w") do io
    for triple in illsdata.ills_triples
        println(io, join(triple, '\t'))
    end
end

open("EA_data/$illpath/train_ques.txt", "w") do io
    for (que, ner, rels) in zip(ill_train_ques, ill_train_ners, ill_train_rels)
        que = replace(que, r", (a|an) .*?[,?]" => "")
        println(io, que, '\t', ner, '\t', join(rels, '\t'))
    end
end

open("EA_data/$illpath/train_data.txt", "w") do io
    for (que, ner, sols) in zip(ill_train_ques, ill_train_ners, ill_train_sols)
        que = replace(que, r", (a|an) .*?[,?]" => "")
        println(io, que, '\t', ner)
        for sol in sols
            println(io, join(sol, '\t'))
        end
        println(io)
    end
end

open("EA_data/$illpath/valid_ques.txt", "w") do io
    for (que, ner) in zip(ill_valid_ques, ill_valid_ners)
        que = replace(que, r", (a|an) .*?[,?]" => "")
        println(io, que, '\t', ner)
    end
end

open("EA_data/$illpath/equivills.txt", "w") do io
    for line in equivills_values
        println(io, join(line, '\t'))
    end
end